In [8]:
import warnings

In [9]:
warnings.filterwarnings('ignore')

In [10]:
import numpy as np
import pandas as pd
df = pd.read_csv('Alcohol_Sales.csv')
df.columns = ['DATE', 'Sales'] 
print(df)
df['Sale_LastMonth'] = df['Sales'].shift(+1)
df['Sale_2Monthsback'] = df['Sales'].shift(+2)
df['Sale_3Monthback'] = df['Sales'].shift(+3)
print(df)
df = df.dropna()
x = df.iloc[:, 2:]
y = df.iloc[:, 1:2]

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3)

from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()
xtrain = scaler_x.fit_transform(xtrain)
xtest = scaler_x.transform(xtest)

scaler_y = MinMaxScaler()
ytrain = scaler_y.fit_transform(ytrain)
ytest = scaler_y.transform(ytest)

# Reshape xtrain and xtest for LSTM input
xtrain = np.reshape(xtrain, (xtrain.shape[0], 1, xtrain.shape[1]))
xtest = np.reshape(xtest, (xtest.shape[0], 1, xtest.shape[1]))

from keras.models import Sequential
from keras.layers import LSTM, Dense

def create_model(input_shape, optimizer='adam'):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=input_shape))
    model.add(Dense(1))  # Regression output
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

# Wrapping the Keras model using KerasRegressor
from scikeras.wrappers import KerasRegressor
model = KerasRegressor(build_fn=create_model, input_shape=(xtrain.shape[1], xtrain.shape[2]), verbose=0)

# Define the grid search parameters
from sklearn.model_selection import GridSearchCV
param_grid = {
    'optimizer': ['Adam', 'Adamax', 'Nadam'],
    'batch_size': [5, 10],
    'epochs': [50, 25]
}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(xtrain, ytrain)

# Summarize the results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


           DATE  Sales
0    1992-01-01   3459
1    1992-02-01   3458
2    1992-03-01   4002
3    1992-04-01   4564
4    1992-05-01   4221
..          ...    ...
320  2018-09-01  12396
321  2018-10-01  13914
322  2018-11-01  14174
323  2018-12-01  15504
324  2019-01-01  10718

[325 rows x 2 columns]
           DATE  Sales  Sale_LastMonth  Sale_2Monthsback  Sale_3Monthback
0    1992-01-01   3459             NaN               NaN              NaN
1    1992-02-01   3458          3459.0               NaN              NaN
2    1992-03-01   4002          3458.0            3459.0              NaN
3    1992-04-01   4564          4002.0            3458.0           3459.0
4    1992-05-01   4221          4564.0            4002.0           3458.0
..          ...    ...             ...               ...              ...
320  2018-09-01  12396         14257.0           12640.0          14583.0
321  2018-10-01  13914         12396.0           14257.0          12640.0
322  2018-11-01  14174         139

2025-01-02 19:44:20.571195: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-02 19:44:20.601598: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-02 19:44:20.607566: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-02 19:44:20.622092: I tensorflow/core/util/port.cc:153

Best: 0.836388 using {'batch_size': 5, 'epochs': 50, 'optimizer': 'Adamax'}
0.827369 (0.023792) with: {'batch_size': 5, 'epochs': 50, 'optimizer': 'Adam'}
0.836388 (0.026274) with: {'batch_size': 5, 'epochs': 50, 'optimizer': 'Adamax'}
0.833987 (0.024256) with: {'batch_size': 5, 'epochs': 50, 'optimizer': 'Nadam'}
0.828378 (0.024845) with: {'batch_size': 5, 'epochs': 25, 'optimizer': 'Adam'}
0.831140 (0.025217) with: {'batch_size': 5, 'epochs': 25, 'optimizer': 'Adamax'}
0.830619 (0.026944) with: {'batch_size': 5, 'epochs': 25, 'optimizer': 'Nadam'}
0.832605 (0.029285) with: {'batch_size': 10, 'epochs': 50, 'optimizer': 'Adam'}
0.828232 (0.024791) with: {'batch_size': 10, 'epochs': 50, 'optimizer': 'Adamax'}
0.821386 (0.025445) with: {'batch_size': 10, 'epochs': 50, 'optimizer': 'Nadam'}
0.822295 (0.031830) with: {'batch_size': 10, 'epochs': 25, 'optimizer': 'Adam'}
0.814553 (0.028487) with: {'batch_size': 10, 'epochs': 25, 'optimizer': 'Adamax'}
0.812883 (0.034343) with: {'batch_size'

In [15]:
# Evaluate the model
loss, mae = model.evaluate(xtest, ytest)
print(f"Test Loss: {loss}, Test MAE: {mae}")

AttributeError: 'KerasRegressor' object has no attribute 'evaluate'